In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("data/melb_data.csv")
y = data.Price
X = data.drop(['Price'],axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

catagorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == 'object']

print(catagorical_cols)
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

my_cols = catagorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

X_train.head()




['Type', 'Method', 'Regionname']


,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


In [30]:
# Define Pre-Processing Steps

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_tranformer = SimpleImputer(strategy='mean') #contant form example. we need to improve

categorical_tranformer = Pipeline(steps=[
        ('imputer' , SimpleImputer(strategy='most_frequent')), # most_frequent
        ('onehot',OneHotEncoder(handle_unknown='ignore'))
    ])
preprocessor = ColumnTransformer(transformers=[
    ('num',numerical_tranformer,numerical_cols),
    ('cat',categorical_tranformer,catagorical_cols)
])



In [31]:
# Define a model
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,random_state=0)

my_pipeline = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])

my_pipeline.fit(X_train,y_train)

pred = my_pipeline.predict(X_valid)

score = mean_absolute_error(y_valid,pred)
print("MAE:",score)


MAE: 160814.63047680413


In [32]:
# Cross validation

from sklearn.model_selection import cross_val_score

score  = -1 * cross_val_score(my_pipeline,X,y,cv=5,scoring='neg_mean_absolute_error')


NameError: name 'scores' is not defined

In [35]:
print("MAE scores:\n", score)
print("Average MAE score (across experiments):")
print(score.mean())

MAE scores:
 [208444.47707413 193158.15832264 186451.75236745 153024.44417158
 156134.16993618]
Average MAE score (across experiments):
179442.60037439512
